In [ ]:
!git clone https://github.com/CTLab-ITMO/self-expanding-nets
%pip install -U -e ./self-expanding-nets/

Cloning into 'self-expanding-nets'...
remote: Enumerating objects: 1099, done.
remote: Counting objects: 100% (231/231), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 1099 (delta 146), reused 140 (delta 69), pack-reused 868 (from 1)
Receiving objects: 100% (1099/1099), 2.43 MiB | 4.03 MiB/s, done.
Resolving deltas: 100% (666/666), done.
Obtaining file:///content/self-expanding-nets
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 

download MultNIST dataset

In [ ]:
!wget https://data.ncl.ac.uk/ndownloader/articles/24574678/versions/1 -O MultNIST.zip
!unzip MultNIST.zip

--2025-05-06 10:56:25--  https://data.ncl.ac.uk/ndownloader/articles/24574678/versions/1
Resolving data.ncl.ac.uk (data.ncl.ac.uk)... 34.241.90.58, 52.49.76.148, 54.195.113.107, ...
Connecting to data.ncl.ac.uk (data.ncl.ac.uk)|34.241.90.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 659123404 (629M) [application/zip]
Saving to: ‘MultNIST.zip’

MultNIST.zip        100%[===================>] 628.59M  21.0MB/s    in 27s     

2025-05-06 10:56:53 (23.2 MB/s) - ‘MultNIST.zip’ saved [659123404/659123404]

Archive:  MultNIST.zip
 extracting: metadata                
 extracting: test_y.npy              
 extracting: train_y.npy             
 extracting: valid_y.npy             
 extracting: test_x.npy              
 extracting: valid_x.npy             
 extracting: train_x.npy             
 extracting: README                  


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, TensorDataset
from torchvision import datasets, transforms

################################
#     RESTART     RUNTIME      #
################################
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [ ]:
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=3 * 28 * 28, output_size=10):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.fc0(x)

In [ ]:
hyperparams = {
    "num_epochs": 128,
    "batch_size": 256,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.4},
    "choose_thresholds_del": {"fc0": 0.04},
    "threshold": 0.05,
    "min_delta_epoch_replace": 20,
    "window_size": 5,
    "lr": 1e-4,
    "delete_after": 4,
    "task_type": "classification",
    "fully_connected": False,
    "max_to_replace": 4000 # None -> no limit
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

In [ ]:
train_x = torch.from_numpy(np.load('/content/train_x.npy'))
train_y = torch.from_numpy(np.load('/content/train_y.npy'))
valid_x = torch.from_numpy(np.load('/content/valid_x.npy'))
valid_y = torch.from_numpy(np.load('/content/valid_y.npy'))
test_x = torch.from_numpy(np.load('/content/test_x.npy'))
test_y = torch.from_numpy(np.load('/content/test_y.npy'))

In [ ]:
train_tensor_x = torch.tensor(train_x).float().view(-1, 3 * 28 * 28)
train_tensor_y = torch.tensor(train_y).long()

valid_tensor_x = torch.tensor(valid_x).float().view(-1, 3 * 28 * 28)
valid_tensor_y = torch.tensor(valid_y).long()

test_tensor_x = torch.tensor(test_x).float().view(-1, 3 * 28 * 28)
test_tensor_y = torch.tensor(test_y).long()

train_dataset = TensorDataset(train_tensor_x, train_tensor_y)
valid_dataset = TensorDataset(valid_tensor_x, valid_tensor_y)
test_dataset = TensorDataset(test_tensor_x, test_tensor_y)

train_loader = DataLoader(train_dataset, batch_size=hyperparams['batch_size'], shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=hyperparams['batch_size'], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=hyperparams['batch_size'], shuffle=False)

<ipython-input-6-a627f8f1a94c>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_tensor_x = torch.tensor(train_x).float().view(-1, 3 * 28 * 28)
<ipython-input-6-a627f8f1a94c>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_tensor_y = torch.tensor(train_y).long()
<ipython-input-6-a627f8f1a94c>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  valid_tensor_x = torch.tensor(valid_x).float().view(-1, 3 * 28 * 28)
<ipython-input-6-a627f8f1a94c>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.cl

In [ ]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0], device=device)

In [ ]:
import wandb
wandb.login()
wandb.finish()
run = wandb.init(
    project="self-expanding-nets-MultNIST",
    name=f"trash",
    config=hyperparams
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vanyamironov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(sparse_model.parameters(), lr=hyperparams['lr'], weight_decay=1e-2)
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, criterion, optimizer, hyperparams, device)

100%|██████████| 196/196 [00:01<00:00, 105.32it/s]


Epoch 1/128, Train Loss: 2.0382, Val Loss: 1.8358, Val Accuracy: 0.2763


100%|██████████| 196/196 [00:01<00:00, 177.63it/s]


Epoch 2/128, Train Loss: 1.8120, Val Loss: 1.7340, Val Accuracy: 0.2945


100%|██████████| 196/196 [00:01<00:00, 136.56it/s]


...

Epoch 102/128, Train Loss: 0.6501, Val Loss: 0.7671, Val Accuracy: 0.7433


100%|██████████| 196/196 [00:41<00:00,  4.69it/s]


Epoch 103/128, Train Loss: 0.6378, Val Loss: 0.7349, Val Accuracy: 0.7581


100%|██████████| 196/196 [00:41<00:00,  4.69it/s]


Epoch 104/128, Train Loss: 0.6294, Val Loss: 0.7358, Val Accuracy: 0.7593


In [ ]:
'fff'

In [ ]:
'sdlfsdkjf'

In [ ]:
'dsklfj'

In [ ]:
_, accuracy = eval_one_epoch(sparse_model, criterion, test_loader, hyperparams['task_type'], device)
params = get_params_amount(sparse_model)

In [ ]:
accuracy, params

In [ ]:
import pandas as pd

data = {
    'Model': ['Ours', 'ResNet-18', 'AlexNet', 'VGG16', 'ConvNext', 'MNASNet', 'DenseNet', 'ResNeXt', 'PC-DARTS', 'DrNAS', 'Bonsai-Net', 'DARTS', ' Bonsai', 'Random'],
    'Accuracy (%)': [accuracy * 100, 91.55, 94.01, 90.43, 64.20, 87.70, 92.81, 90.57, 96.68, 98.10, 97.17, 96.55, 39.76, 10],
    'Parameters': [params, 11_689_512, 61_100_840, 138_357_544, 88_591_464, 4_383_312, 28_681_000, 25_028_904]
}

table = pd.DataFrame(data)

def format_with_commas(x):
    return "{:,}".format(x)

styled_table = (table.style
               .format({'Accuracy (%)': '{:.2f}',
                       'Parameters': format_with_commas})
               .set_properties(**{'text-align': 'center'})
               .set_table_styles([
                   {'selector': 'th', 'props': [('text-align', 'center')]},
                   {'selector': 'caption', 'props': [('font-size', '1.1em')]}
               ])
               .hide(axis='index'))

styled_table